In [1]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [2]:
#path = DATA_DIR/'sample_gtfs.zip'
#path = DATA_DIR/'other_feeds'/'gtfs.zip'
path = DATA_DIR/'cairns_gtfs.zip'

print(gt.list_gtfs(path))

feed = gt.read_gtfs(path, dist_units='km')


            file_name  file_size
0  calendar_dates.txt        387
1          routes.txt       1478
2           trips.txt     143081
3           stops.txt      26183
4          agency.txt        199
5      stop_times.txt    2561019
6          shapes.txt     864694
7        calendar.txt        337


In [29]:
dates = feed.get_first_week()[0:1]
feed1 = feed.copy()
del feed1.trips["direction_id"]
trip_stats = feed1.compute_trip_stats().reset_index(drop=True)

f = trip_stats.copy()
#f.loc[0, "direction_id"] = 0
#f.loc[1, "direction_id"] = 1

feed1.compute_route_time_series(f, dates=dates, split_directions=False, freq="12H").T


2014-05-26 00:00:00  2014-05-26 12:00:00
indicator        route_id                                          
num_trip_ends    110-423             19.000000            39.000000
                 111-423             18.000000            39.000000
                 112-423              4.000000            11.000000
                 113-423              3.000000             3.000000
                 120-423             11.000000            21.000000
                 120N-423             0.000000             2.000000
                 121-423             13.000000            21.000000
                 122-423             13.000000            20.000000
                 123-423             21.000000            38.000000
                 130-423             11.000000            22.000000
                 131-423             10.000000            22.000000
                 131N-423             0.000000             1.000000
                 133-423             11.000000            24.000000
                 140-423             16.000000            23.000000
                 141-423             19.000000            28.000000
                 142-423             15.000000            27.000000
                 143-423             19.000000            29.000000
                 143W-423             0.000000             9.000000
                 150-423             11.000000            16.000000
                 150E-423             0.000000             8.000000
num_trip_starts  110-423             23.000000            36.000000
                 111-423             22.000000            36.000000
                 112-423              5.000000            10.000000
                 113-423              3.000000             3.000000
                 120-423             12.000000            20.000000
                 120N-423             0.000000             2.000000
                 121-423             14.000000            20.000000
                 122-423             14.000000            19.000000
                 123-423             23.000000            37.000000
                 130-423             12.000000            21.000000
...                                        ...                  ...
service_duration 131-423              5.600000            10.933333
                 131N-423             0.000000             0.516667
                 133-423              7.383333            14.816667
                 140-423             15.466667            20.300000
                 141-423             13.366667            17.166667
                 142-423             13.766667            24.283333
                 143-423             16.000000            20.866667
                 143W-423             0.000000             6.750000
                 150-423             12.283333            15.150000
                 150E-423             0.000000             8.933333
service_speed    110-423             32.243592            33.483585
                 111-423             33.070961            34.127874
                 112-423             35.269098            35.269098
                 113-423             35.663218            39.623348
                 120-423             33.766810            33.751118
                 120N-423                  NaN            47.649553
                 121-423             32.205455            32.399211
                 122-423             35.454454            35.001354
                 123-423             27.435700            28.135631
                 130-423             21.174730            21.176642
                 131-423             24.049902            24.051800
                 131N-423                  NaN            24.067374
                 133-423             25.035081            26.439134
                 140-423             25.634669            25.832253
                 141-423             20.802088            20.828188
                 142-423             26.021598            26.417789
               

In [12]:
def compute_route_stats_split_directions(group):
    # Take this group of all trips stats for a single route
    # and compute route-level stats.
    d = OrderedDict()
    d["route_short_name"] = group["route_short_name"].iat[0]
    d["route_type"] = group["route_type"].iat[0]
    d["num_trips"] = group.shape[0]
    d["num_trip_starts"] = group["start_time"].count()
    d["is_loop"] = int(group["is_loop"].any())
    d["start_time"] = group["start_time"].min()
    d["end_time"] = group["end_time"].max()

    d["service_distance"] = group["distance"].sum()
    d["service_duration"] = group["duration"].sum()

    return pd.Series(d)

f.groupby(["route_id", "direction_id"]).apply(compute_route_stats_split_directions).reset_index()


,route_id,direction_id,route_short_name,route_type,num_trips,num_trip_starts,is_loop,start_time,end_time,service_distance,service_duration
0,110-423,0.0,110,3,1,1,0,05:50:00,06:50:00,32.507121,1.0
1,110-423,1.0,110,3,1,1,0,06:16:00,07:10:00,32.507121,0.9


In [ ]:
dates = feed.get_first_week()[6:]
feed1 = feed.restrict_to_dates(["20180101"])
feed1.routes


In [ ]:
#f = feed.compute_stop_time_series(dates[:3], split_directions=True, freq='12H')
#f = feed.compute_route_time_series(trip_stats, dates[:1], freq='12H', split_directions=True)
f = feed.compute_feed_time_series(trip_stats, dates[:1], freq='12H')
#f.columns
pd.isnull(f.values).all()

In [ ]:
ts = feed.compute_route_time_series(trip_stats, dates[0], freq='6H')
# inds = ['num_trips', 'num_trip_ends', 'num_trip_starts']
# rids = ['110-423', '111-423', '112-423']

# ts.loc[:, (inds, rids)]

#ts = feed.compute_feed_time_series(trip_stats, [dates[0], dates[2]], freq='6H')
# ts = feed.compute_stop_time_series([dates[0], dates[2]], freq='6H')
# sid = '750015'
# ts.loc[:, ('num_trips', sid)]

ts

In [ ]:
ts.columns

In [ ]:
dts = gt.downsample(ts, freq='D')
#dts.loc[:, (inds, rids)]
#dts.loc[:, ('num_trips', sid)]
dts

In [ ]:
f = feed.compute_feed_stats(trip_stats, '2001')
f


In [ ]:
f.date

In [ ]:
feed.compute_stop_stats(['20140601', '20140603'])

In [ ]:
trip_stats = feed.compute_trip_stats()
feed = feed.append_dist_to_stop_times(trip_stats)

# Load screen line
with (DATA_DIR/'cairns_screen_line.geojson').open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])



In [ ]:
f = feed.compute_screen_line_counts(line, dates[:7])
f[f['crossing_time'] < '06:00:00']

In [ ]:
feed.compute_feed_time_series(ts, dates[-2:], freq='12H')

In [ ]:
sd = False
#%time p1 = proto1(feed, ts, dates[:14], split_directions=sd)
%time p2 = proto2(feed, [dates[0], '20010101'], split_directions=sd, freq='12H')
p2

In [ ]:
feed.compute_feed_stats(ts, dates[0])

In [ ]:
feed.describe()

In [ ]:
feed.summarize()